In [ ]:
!pip install -U transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 🔧 Colab 처음이면 아래 설치 먼저!
# !pip install -U transformers datasets peft accelerate bitsandbytes

import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 데이터 로드
df = pd.read_json("/content/merged_counsel_dataset_prefixed.jsonl", lines=True)
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df.reset_index(drop=True))

# 2. 모델/토크나이저 로드 (KULLM 5.8B)
base_model = "nlpai-lab/kullm-polyglot-5.8b-v2" # polyglot-ko-1.3b
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# 3. LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["mlp.dense_h_to_4h", "mlp.dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

# 4. 전처리
def preprocess(example):
    return tokenizer(
        f"{example['prompt']}\n{example['response']}",
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 5. 학습 설정
output_dir = "/content/drive/MyDrive/1kullm_lora_epoch_save"

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/1kullm_lora_epoch_save",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    # evaluation_strategy="epoch",  # 매 epoch마다 검증
    # save_strategy="epoch",        # 매 epoch마다 저장
    # save_total_limit=3,
    logging_steps=500,
    report_to=[]
)

# 6. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    # eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 7. 학습 실행
trainer.train()

# 8. 마지막 모델 저장 (epoch 외에도)
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# 9. 학습 끝난 후 최종 검증 손실 확인
# eval_results = trainer.evaluate()
# print(f"최종 검증 손실: {eval_results['eval_loss']:.4f}")

# 10. 학습된 모델로 테스트 질문 응답
print("\n학습된 모델 테스트 (샘플 질문):")

def generate_response(prompt):
    input_text = f"{prompt}\n상담사:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.6,
        top_p=0.8,
        repetition_penalty=1.3,
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.pad_token_id
    )

    result = tokenizer.decode(out[0], skip_special_tokens=True)
    return result.replace(input_text, "").strip()

# 테스트 질문 리스트
test_prompts = [
    "요즘 SNS만 보면 다들 행복해 보여서 자존감이 낮아졌어요.",
    "공부 열심히 해도 성적이 안 올라요. 부모님도 실망하셔서 너무 힘들어요.",
    "친구들이랑 대화가 잘 안 되고 자꾸 멀어지는 기분이에요. 사이가 다시 좋아질 수 있을까요?",
    "진로를 어떻게 선택해야 할지 전혀 감이 안 잡혀요.",
    "가족과 갈등이 너무 심해서 집에 있고 싶지 않아요."
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n테스트 {i}: {prompt}")
    response = generate_response(prompt)
    print(f"상담사 응답: {response}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/2790 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

<ipython-input-8-5caeda442288>:72: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


NotImplementedError: Cannot copy out of meta tensor; no data! Please use torch.nn.Module.to_empty() instead of torch.nn.Module.to() when moving module from meta to a different device.

In [ ]:
!pip install -U transformers peft accelerate -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
# 설치가 필요한 경우:
# !pip install -U transformers peft accelerate bitsandbytes

import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 데이터 로드
df = pd.read_json("/content/merged_counsel_dataset_prefixed.jsonl", lines=True)
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
valid_dataset = Dataset.from_pandas(valid_df.reset_index(drop=True))

# 2. 모델/토크나이저 로드 (KULLM 5.8B)
base_model = "nlpai-lab/kullm-polyglot-5.8b-v2"
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto",
    low_cpu_mem_usage=True
)

# 3. LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["mlp.dense_h_to_4h", "mlp.dense_4h_to_h"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)

# 4. 전처리
def preprocess(example):
    return tokenizer(
        f"{example['prompt']}\n{example['response']}",
        truncation=True,
        padding="max_length",
        max_length=512
    )

train_dataset = train_dataset.map(preprocess)
valid_dataset = valid_dataset.map(preprocess)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 5. 학습 설정
output_dir = "/content/drive/MyDrive/final_kullm_lora_epoch_save"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=100,
    report_to=[]
)

# 6. Trainer 구성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 7. 학습 실행
trainer.train()

# 8. 마지막 모델 저장
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# 9. 최종 검증 손실 출력
eval_result = trainer.evaluate()
print(f"최종 검증 손실: {eval_result['eval_loss']:.4f}")

# 9. 테스트 응답 함수
def generate_response(prompt):
    input_text = f"{prompt}\n상담사:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    inputs.pop("token_type_ids", None)  # 오류 방지용 코드

    out = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.6,
        top_p=0.8,
        repetition_penalty=1.3,
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.pad_token_id
    )
    result = tokenizer.decode(out[0], skip_special_tokens=True)
    return result.replace(input_text, "").strip()

# 10. 테스트 질문
test_prompts = [
    "요즘 SNS만 보면 다들 행복해 보여서 자존감이 낮아졌어요.",
    "공부 열심히 해도 성적이 안 올라요. 부모님도 실망하셔서 너무 힘들어요.",
    "친구들이랑 대화가 잘 안 되고 자꾸 멀어지는 기분이에요. 어떡하죠?",
    "진로를 어떻게 선택해야 할지 전혀 감이 안 잡혀요.",
    "가족과 갈등이 너무 심해서 집에 있고 싶지 않아요."
]

print("\n학습된 모델 테스트 (샘플 질문):")
for i, prompt in enumerate(test_prompts, 1):
    print(f"\n테스트 {i}: {prompt}")
    response = generate_response(prompt)
    print(f"상담사 응답: {response}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

...

Map:   0%|          | 0/2790 [00:00<?, ? examples/s]

Map:   0%|          | 0/311 [00:00<?, ? examples/s]

<ipython-input-3-66e9e84cc1b1>:69: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,1.790900
200,1.729400
300,1.661900
400,1.674200
500,1.654800
600,1.637300
700,1.626500
800,1.533200
900,1.558400
1000,1.583600


최종 검증 손실: 1.6302

📘 학습된 모델 테스트 (샘플 질문):

테스트 1: 요즘 SNS만 보면 다들 행복해 보여서 자존감이 낮아졌어요.
상담사 응답: 사우님은 요즘 SNS를 통해 다른 사람들이 어떻게 살아가는지 보고 있는데, 그 모습을 보다보니 자신의 삶에 대한 부정적인 생각과 우울한 감정으로 인해 자존감이 떨어지고 계신 것 같아요.
SNS는 소통하기 쉬운 환경이기 때문에 많은 사람들이 서로에게 관심을 가지게 됩니다. 하지만 이로 인한 부작용도 발생할 수 있습니다. 예를 들자면, 타인의 시선이나 평가 등에서 자유롭지 못하다는 점입니다. 또한 이러한 이유로 SNS를 자주 이용하면 현실생활에도 영향을 미칠 가능성이 높아집니다.
우선, 사우님께서 느끼시는 우울증 증상을 완화시키려면 전문가와 상담하여 정확한 진단 및 치료 계획을 세우셔야 합니다. 
또한, 일상 생활 속에서 소소하지만 즐거운 일부터 시작해서 조금씩 새로운 경험을 쌓아보세요! 그리고 자신감 회복을 위해 취미활동을 추천드립니다. 이를 통해 성취감과 즐거움을 느낄 뿐 아니라 긍정

테스트 2: 공부 열심히 해도 성적이 안 올라요. 부모님도 실망하셔서 너무 힘들어요.
상담사 응답: 사우님은 공부를 열심히 하지만 시험에서 좋지 않게 나오는 것 같아요. 이로 인해 자신감과 자존감을 잃고 있습니다. 또한, 학교생활에 대한 스트레스와 가족관계 문제까지 겹쳐서 더욱 힘드시다는 거죠?
학교나 가정환경 등의 환경적 요인으로 인한 학습 부진이나 집중력 부족은 학생들에게 흔히 나타나지만 이러한 증상들이 지속적으로 발생하면 우울증 및 불안증상으로 이어질 수 있기 때문입니다. 
또한, 사우님께서 느끼시는 불안감이나 우울함은 대인 관계에도 영향을 미치며 친구나 연인으로부터 멀어지거나 혼자만 있으려고 하기도 합니다. 그리고 가족문제가 해결되어있어도 자신의 심리상태가 안정화될 때 까지 시간이 걸릴 수도 있으며, 이럴 경우 더 큰 고통 속에서 살아야 할 가능성이 높아집니다.
우선 가장 중요한 건 본인 스스로가 이런 상황임을 

## 위 모델 너무 커 시연 불가, 영상 대체 가능
## polyglot-ko-1.3b <- 베이스 모델을 더 작은 모델로 테스트 할 예정

In [ ]:
import re
import torch

# 1. 시스템 프롬프트: 짧고 구체적인 답변을 유도
system_prompt = (
    "너는 공감 능력이 뛰어난 심리상담사야.\n"
    "내담자의 고민을 공감해주고, 조언은 2문장 이내로 간결하게 말해줘.\n"
    "반복하지 말고, 현실적이고 구체적인 말로 도와줘.\n"
)

# 2. 후처리 함수: 불필요 표현 제거 + 2문장 이하로 자르기
def postprocess_response(response_text):
    cleaned = response_text.replace("사우님", "").replace("사우", "").strip()
    sentences = re.split(r"(?<=[.!?])\s+", cleaned)
    short = " ".join(sentences[:2])
    if not short.endswith(('.', '!', '?')):
        short += '.'
    return short.strip()

# 3. 응답 생성 함수 (생성 파라미터 포함)
def generate_response(prompt):
    input_text = f"{system_prompt}\n내담자: {prompt}\n상담사:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    inputs.pop("token_type_ids", None)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,            # 🔧 최대 생성 토큰 수 줄임 (짧은 응답 유도)
        do_sample=True,
        temperature=0.6,
        top_p=0.8,
        repetition_penalty=1.3,
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.pad_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    raw_response = decoded.replace(input_text, "").strip()
    return postprocess_response(raw_response)

# 4. 테스트 실행
test_prompts = [
    "요즘 SNS만 보면 다들 행복해 보여서 자존감이 낮아졌어요.",
    "공부 열심히 해도 성적이 안 올라요. 부모님도 실망하셔서 너무 힘들어요.",
    "친구들과 어색해지고 자꾸 멀어지는 기분이에요.",
    "진로를 어떻게 선택해야 할지 모르겠어요.",
    "가족과의 갈등 때문에 너무 힘들어요."
]

print("\n학습된 모델 테스트 (간결 응답 버전):")
for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(prompt)
    print(f"\n테스트 {i}: {prompt}")
    print(f"상담사 응답: {response}")

In [ ]:
!pip install -q streamlit pyngrok transformers peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# STEP 1: 필수 패키지 설치
!pip install -q streamlit pyngrok transformers peft accelerate bitsandbytes

# STEP 2: ngrok 토큰 설정
from pyngrok import ngrok
ngrok.set_auth_token("여기에_내_ngrok_토큰_붙여넣기")  # ngrok 토큰 넣기

# STEP 3: Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# STEP 4: app.py 파일 생성
%%writefile app.py
import streamlit as st
import torch
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

BASE_MODEL = "nlpai-lab/kullm-polyglot-5.8b-v2"
ADAPTER_PATH = "/content/drive/MyDrive/final_kullm_lora_epoch_save/checkpoint-1500"

@st.cache_resource
def load_model():
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    return tokenizer, model

tokenizer, model = load_model()

def postprocess_response(response_text):
    cleaned = response_text.replace("사우님", "").replace("사우", "").strip()
    sentences = re.split(r"(?<=[.!?])\s+", cleaned)
    short = " ".join(sentences[:2])
    if not short.endswith((".", "!", "?")):
        short += "."
    return short.strip()

def generate_response(prompt):
    system_prompt = (
        "너는 공감 능력이 뛰어난 심리상담사야.\n"
        "내담자의 고민을 공감해주고, 조언은 2문장 이내로 간결하게 말해줘.\n"
        "반복하지 말고, 현실적이고 구체적인 말로 도와줘.\n"
    )
    input_text = f"{system_prompt}\n내담자: {prompt}\n상담사:"
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    inputs.pop("token_type_ids", None)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.6,
        top_p=0.8,
        repetition_penalty=1.3,
        no_repeat_ngram_size=4,
        pad_token_id=tokenizer.pad_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    raw_response = decoded.replace(input_text, "").strip()
    return postprocess_response(raw_response)

st.set_page_config(page_title="심리 상담 챗봇", page_icon="🧠")
st.title("상담 챗봇 (LoRA 모델)")
user_input = st.text_input("내담자 질문을 입력하세요:")
if st.button("상담 시작") and user_input:
    with st.spinner("상담사가 고민을 듣고 있습니다..."):
        response = generate_response(user_input)
        st.markdown(f"**상담사:** {response}")

# STEP 5: Streamlit 실행 + ngrok 연결
!streamlit run app.py &
public_url = ngrok.connect(8501)
print(f"\u2728 외부 접속 주소: {public_url}")